<a href="https://colab.research.google.com/github/weilipan/MachineLearing/blob/main/1_2_%E9%81%BA%E6%BC%8F%E5%80%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1-2-1 偵測與刪除遺漏值
處理真實數據時常會遇到數據不完整或數據遺漏的情況，遺漏值除了容易造成分析結果的偏誤外，也難以直接交由機器學習模型來擬合。

In [2]:
#下載寶可夢檔案
!wget pokemon.csv https://gist.githubusercontent.com/armgilles/194bcff35001e7eb53a2a8b441e8b2c6/raw/92200bc0a673d5ce2110aaad4544ed6c4010f687/pokemon.csv

--2023-03-30 02:22:42--  http://pokemon.csv/
Resolving pokemon.csv (pokemon.csv)... failed: Name or service not known.
wget: unable to resolve host address ‘pokemon.csv’
--2023-03-30 02:22:42--  https://gist.githubusercontent.com/armgilles/194bcff35001e7eb53a2a8b441e8b2c6/raw/92200bc0a673d5ce2110aaad4544ed6c4010f687/pokemon.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 44028 (43K) [text/plain]
Saving to: ‘pokemon.csv’

pokemon.csv         100%[===================>]  43.00K  --.-KB/s    in 0.005s  

2023-03-30 02:22:42 (7.99 MB/s) - ‘pokemon.csv’ saved [44028/44028]

FINISHED --2023-03-30 02:22:42--
Total wall clock time: 0.3s
Downloaded: 1 files, 43K in 0.005s (7.99 MB/s)


In [3]:
# 透過pandas的info()顯示的一覽表
import numpy as np
import pandas as pd
df=pd.read_csv('pokemon.csv',)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   #           800 non-null    int64 
 1   Name        800 non-null    object
 2   Type 1      800 non-null    object
 3   Type 2      414 non-null    object
 4   Total       800 non-null    int64 
 5   HP          800 non-null    int64 
 6   Attack      800 non-null    int64 
 7   Defense     800 non-null    int64 
 8   Sp. Atk     800 non-null    int64 
 9   Sp. Def     800 non-null    int64 
 10  Speed       800 non-null    int64 
 11  Generation  800 non-null    int64 
 12  Legendary   800 non-null    bool  
dtypes: bool(1), int64(9), object(3)
memory usage: 75.9+ KB


檢視一覽表可知總共有12個特徵和800個樣本，其中Type 2遺漏值386筆。遺漏值用字串NA與空白來表示，可以透過isnull()與isna()兩個方法。但是要設定某筆數據為遺漏值則要藉由numpy套件以np.nan的方式處理，如果是時間的遺漏值則是用pandas套件pd.NaT。

In [4]:
df[df['Type 2'].isna()] 

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
5,5,Charmeleon,Fire,NaN,405,58,64,58,80,65,80,1,False
9,7,Squirtle,Water,NaN,314,44,48,65,50,64,43,1,False
10,8,Wartortle,Water,NaN,405,59,63,80,65,80,58,1,False
11,9,Blastoise,Water,NaN,530,79,83,100,85,105,78,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
775,705,Sliggoo,Dragon,NaN,452,68,75,53,83,113,60,6,False
776,706,Goodra,Dragon,NaN,600,90,100,70,110,150,80,6,False
788,712,Bergmite,Ice,NaN,304,55,69,85,32,35,28,6,False
789,713,Avalugg,Ice,NaN,514,95,117,184,44,46,28,6,False


In [5]:
# 對付遺漏值最簡單方式是刪除有遺漏數的樣本或特徵，要注意的是當樣本或特徵數過少時，
# 直接刪除的作法容易造成後續模型的效能低落。
df_drop=df.dropna() #刪除所有含nan的列
# df_drop=df.dropna(thresh=11) #刪除非遺漏值小於11個的列
df_drop.shape

(414, 13)

In [7]:
df_type2=df[df['Type 2'].notna()]
df_type2.shape

(414, 13)

## 填補遺漏值
1. 完全隨機遺漏(Missing Completely at Random、MCAR)：數據的遺漏純粹為隨機發生，此時可刪除該筆觀察值或其他變數推估。
2. 隨機遺漏(Missing at Random、MAR)：數據的遺漏不受自身變數的影響，但會受到其他變數的影響，此時可利用平均數、中位數或眾數等統計量進行填補。
3. 非隨機遺漏(Missing Not at Random、MNAR)：數據的遺漏非由隨機因素產生，可能是受訪者對敏感的問題刻意不答，或是問卷太過冗長而漏答，這種遺漏值的發生原因需要進一步分析，而若某一個特徵有太多遺漏值，也可考慮逕行刪除。

In [10]:
df[df['Type 2'].isna()]

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
5,5,Charmeleon,Fire,NaN,405,58,64,58,80,65,80,1,False
9,7,Squirtle,Water,NaN,314,44,48,65,50,64,43,1,False
10,8,Wartortle,Water,NaN,405,59,63,80,65,80,58,1,False
11,9,Blastoise,Water,NaN,530,79,83,100,85,105,78,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
775,705,Sliggoo,Dragon,NaN,452,68,75,53,83,113,60,6,False
776,706,Goodra,Dragon,NaN,600,90,100,70,110,150,80,6,False
788,712,Bergmite,Ice,NaN,304,55,69,85,32,35,28,6,False
789,713,Avalugg,Ice,NaN,514,95,117,184,44,46,28,6,False


In [9]:
df.fillna(0).tail(2)

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True
799,721,Volcanion,Fire,Water,600,80,110,120,130,90,70,6,True


In [11]:
# scikit-learn也提供方便的填補器SimpleImputer，能填補平均值、中位數、眾數與常數。
from sklearn.impute import SimpleImputer
# 用眾數進行填補，好處是能填補非數值型數據
imp=SimpleImputer(missing_values=np.nan,strategy='most_frequent')
data=imp.fit_transform(df) #填補後回傳陣列
df_imp=pd.DataFrame(data,columns=df.columns)
df_imp.tail(3)

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True
799,721,Volcanion,Fire,Water,600,80,110,120,130,90,70,6,True


In [12]:
# KNN機器學習
from sklearn.impute import KNNImputer
imp=KNNImputer(n_neighbors=5)
data=imp.fit_transform(df.loc[:,'HP':'Speed'])
df_imp=df
df_imp.loc[:,'HP':'Speed']=data
df_imp.tail(2)

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80.0,160.0,60.0,170.0,130.0,80.0,6,True
799,721,Volcanion,Fire,Water,600,80.0,110.0,120.0,130.0,90.0,70.0,6,True
